In [5]:
import omop_alchemy as oa
from omop_alchemy.db.populate_db import to_load_vocabulary, to_load_clinical, to_load_health_system, rapid_load
import csv
from pathlib import Path

In [29]:
path = Path('/Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/ohdsi_vocabs')
target = to_load_vocabulary
rapid_load_script = 'SET session_replication_role = \'replica\';'
for f in path.iterdir(): 
    if f.name in target:

        with open(f, 'r') as file:
            reader = csv.DictReader(file, delimiter='\t')
            for row in reader:
                header = row
                break
        cols = '\',\''.join(list(header.keys()))
        rapid_load_script += f'\n\copy {f.stem.lower()} (\'{cols}\') from {f} delimiter E\'\\t\' CSV HEADER QUOTE \'~\''


In [30]:
# you will need to stop and restart the kernel if it's your first time populating concepts in the DB as the vocab lookups will not populate correctly, but 
# that is OK because it is expected that the reference vocabs will be in place before trying to load any clinical data

print(rapid_load_script)

SET session_replication_role = 'replica';
\copy concept_relationship ('concept_id_1','concept_id_2','relationship_id','valid_start_date','valid_end_date','invalid_reason') from /Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/ohdsi_vocabs/CONCEPT_RELATIONSHIP.csv delimiter E'\t' CSV HEADER QUOTE '~'
\copy vocabulary ('vocabulary_id','vocabulary_name','vocabulary_reference','vocabulary_version','vocabulary_concept_id') from /Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/ohdsi_vocabs/VOCABULARY.csv delimiter E'\t' CSV HEADER QUOTE '~'
\copy concept_class ('concept_class_id','concept_class_name','concept_class_concept_id') from /Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/ohdsi_vocabs/CONCEPT_CLASS.csv delimiter E'\t' CSV HEADER QUOTE '~'
\copy concept ('concept_id','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept','concept_code','valid_start_date','valid_end_d

In [ ]:
# make sure to download a small athena zip file (suggested no more than 2-4 vocabs for demo purposes)
# and unzip to omop_alchemy/resources/ohdsi_vocabs before running this script. 

populate_demo_db(to_load_health_system)

In [ ]:

populate_demo_db(to_load_clinical)

In [ ]:

import sqlalchemy.orm as so
import sqlalchemy.sql.sqltypes as sss
from omop_alchemy.tables.clinical import Person, Condition_Occurrence, Measurement, Observation

with so.Session(oa.oa_config.engine) as sess:
    people = sess.query(Person
                        ).options(so.joinedload(Person.conditions, 
                                                Condition_Occurrence.condition_concept)
                        ).options(so.joinedload(Person.conditions,
                                                Condition_Occurrence.modifiers,
                                                Measurement.measurement_concept)
                        ).options(so.joinedload(Person.conditions,
                                                Condition_Occurrence.related_obs,
                                                Observation.observation_concept)).all()

In [ ]:
[[(c.condition_concept.concept_name, [m.measurement_concept for m in c.modifiers]) for c in p.conditions] for p in people]

In [ ]:
[[(c.condition_concept.concept_name, [m.observation_concept for m in c.related_obs]) for c in p.conditions] for p in people]

In [ ]:
from omop_alchemy.tables.clinical import Person, Condition_Occurrence, Measurement, Observation
from omop_alchemy.tables.vocabulary import Concept, Concept_Relationship

Regimen = so.aliased(Concept, name='regimen')
Drugs = so.aliased(Concept, name='drugs')

with so.Session(oa.oa_config.engine) as sess:

        query = sess.query(Regimen.concept_id.label('regimen_id'),
                          Regimen.concept_name.label('regimen_name'),
                          Concept_Relationship.relationship_id,
                          Drugs.concept_id.label('drug_id'),
                          Drugs.concept_name.label('drug_name')
                          ).join(Concept_Relationship, Concept_Relationship.concept_id_1==Regimen.concept_id
                          ).join(Drugs, Concept_Relationship.concept_id_2==Drugs.concept_id
                          ).filter(Regimen.vocabulary_id=='HemOnc'
                          ).filter(Regimen.concept_class_id=='Regimen'
                          ).filter(Drugs.vocabulary_id=='HemOnc'
                          ).filter(Drugs.concept_class_id=='Component'
                          ).order_by(Regimen.concept_id.desc())
        r = query.all()

In [ ]:
r